# Imports

In [2]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------

# Funções

## Carregamento modelo

In [3]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


## Inferencia

In [4]:
def inference(model, text):
    text_token = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors='pt'
    )
    text_token = text_token.to('cuda:0')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            input_ids = text_token.input_ids, 
            max_new_tokens=300,
            # temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

# Executando

In [5]:
model_name = 'tiiuae/falcon-7b-instruct'
model, peft_config, tokenizer = create_and_prepare_model(model_name)
model = PeftModel.from_pretrained(model, './results/qlora_falcon7b2023-07-12T16:42:38.435454/checkpoint-100')
gen_input = lambda x: f'''\
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
{x}
###
'''               

Loading checkpoint shards: 100%|██████████| 2/2 [00:33<00:00, 16.92s/it]


In [6]:
model.config.use_cache=False

## Inferencias

In [6]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Alimentação caríssima para um péssimo atendimento.. super mal atendido
###
Caro cliente,

Lamentamos profundamente pela má experiência que você teve em nosso restaurante. Reconhecemos que houve falhas no atendimento e na alimentação, o que resultou em um péssimo sabor à boca.

Agradecemos por compartilhar sua opinião e gostaríamos de esclarecer que estamos tomando medidas para melhorar nossos serviços e garantir que isso não se repita. Pedimos desculpas pelo inconveniente causado e gostaríamos de ter a oportunidade de recebê-lo novamente em nosso restaurante para proporcionar uma experiência melhor.

Atenciosamente,
A equipe do restaurante
###
Caro cliente,

Agradecemos por compartilhar sua experiência conosco e lamentamos profundamente pelos problemas que você enfrentou durante sua visita. Reconhecemos que houve falhas no atendimento e na alimentação, o que resultou em uma má im

In [7]:
comment='Filé estava duro. Ficamos decepcionados.'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Filé estava duro. Ficamos decepcionados.
###
Caro cliente,

Lamentamos muito que a experiência em nosso restaurante não tenha atendido às suas expectativas. Pedimos desculpas pelo filé que foi duro e não atendeu às suas referências. Vamos revisar nossos processos para garantir que isso não aconteça novamente.

Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-lo novamente em nosso estabelecimento para oferecer uma experiência melhor.

Atenciosamente,
A equipe do restaurante
###
Caro cliente,

Agradecemos pelo seu feedback e lamentamos profundamente pela má experiência que você teve em nosso restaurante. Reconhecemos que a qualidade do filé não atendeu às suas expectativas e entendemos a frustração causada pelo aspecto e sabor do prato.

Nós levamos muito a sério a qualidade dos nossos alimentos e vamos revisar nossos processos para garantir que iss

In [8]:
comment='''A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.'''
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.
###
Caro cliente,

Lamentamos profundamente pela falta de assistência e o tratamento inadequado que você e seu amigo tiveram que enfrentar em nosso restaurante. Reconhecemos que isso é totalmente inaceitável e contra nosso padrão de atendimento.

Levaremos suas críticas a sério e tomaremos medidas para melhorar nossos processos e garantir que todos os clientes sejam tratados com igual tratamento, independentemente de sua identidade física.

Pedimos desculpas pelo constrangimento que

In [9]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
###
Prezada cliente,

Lamentamos profundamente pela má experiência que você teve em nosso restaurante. Reconhecemos que houve uma falha no atendimento e no tempo de espera, o que resultou em uma percepção negativa do nosso serviço.

Levaremos suas críticas em consideração para melhorar nossos processos e garantir que isso não se repita. Pedimos desculpas pelo frango com o gosto de peixe e vamos investigar o ocorrido.

Valorizamos a sua opinião e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante, para que possamos oferecer uma experiência mais satisfatória.

Atenciosamente,
A equipe do restaurante
###
Prezada cliente,

Agradecemos por compartilhar sua experiência conosco. Lamentamos profundamente pelo ocorrido e entendemos a sua frustração ao se senti

In [10]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
###
Prezada cliente,

Lamentamos profundamente pela má experiência que você teve em nosso restaurante. Reconhecemos que houve uma falha no atendimento e no tempo de espera, o que resultou em uma percepção negativa do nosso serviço.

Levaremos suas críticas em consideração para melhorar nossos processos e garantir que isso não se repita. Pedimos desculpas pelo frango com o gosto de peixe e vamos investigar o ocorrido.

Valorizamos a sua opinião e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante, para que possamos oferecer uma experiência mais satisfatória.

Atenciosamente,
A equipe do restaurante
###
Prezada cliente,

Agradecemos por compartilhar sua experiência conosco. Lamentamos profundamente pelo ocorrido e entendemos a sua frustração ao se senti

In [12]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
###
Caro cliente,

Agradecemos pelo seu feedback e lamentamos que sua experiência em nosso restaurante não tenha sido satisfatória. Levaremos em consideração suas observações sobre a temperatura do local e o tempo de espera pela comida. Vamos trabalhar para melhorar esses pontos e garantir que isso não ocorra novamente.

Valorizamos muito a opinião dos nossos clientes e estamos empenhados em oferecer uma experiência agradável para todos. Esperamos ter a oportunidade de recebê-lo novamente em nosso estabelecimento para que possamos mostrar a você a qualidade e o cuidado que caracterizam nosso restaurante.

Atenciosamente,
A equipe do restaurante
###
Olá,

Agradecemos pelo seu feedback e lamentamos que a sua experiência em nosso restaurante não tenha sido satisfatória. Levaremos em considera

In [13]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
###
Caro cliente,

Agradecemos pelo seu feedback e lamentamos pela má experiência que você teve em relação ao atendimento no nosso restaurante. Reconhecemos que houve uma falha no atendimento no final dos fins de semana, o que resultou em uma fila de espera longa para os clientes externos.

Estamos trabalhando para melhorar nosso atendimento e garantir que todos os clientes sejam atendidos de maneira eficiente, seja dentro ou fora do estabelecimento. Pedimos desculpas pelo inconveniente causado e vamos avaliar nossos processos internos para evitar que isso ocorra novamente.

Agradecemos por compartilhar sua experiência conosco e esperamos ter a oportunidade de recebê-lo novamente em nosso restau

In [11]:
comment = 'O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!
###
Caro cliente,

Lamentamos muito que você não tenha gostado de nosso yakisoba. Entendemos sua insatisfação e gostaríamos de melhorar a qualidade de nossos pratos. Vamos revisar nossos processos de preparo para garantir que isso não aconteça novamente.

Agradecemos por compartilhar sua experiência conosco e esperamos ter a oportunidade de recebê-lo novamente em nosso restaurante.

Atenciosamente,
A equipe do restaurante
###
Olá,

Agradecemos pelo seu feedback e lamentamos sinceramente pela má experiência que você teve com nosso yakisoba. Reconhecemos que o aspecto e o sabor não atenderam às suas expectativas.

Nós entendemos a importância de oferecer pratos de qualidade e vamos revisar nossos processos de preparo par

In [7]:
comment= '''Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.'''
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento.
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.
###
Olá,

Lamentamos muito pela má experiência que você teve em nosso restaurante. Pedimos desculpas pelo atraso na entrega do seu pedido e pela falta de preparação da porção de omelete. Vamos reforçar com nossa equipe a importância de evitar esse tipo de falha no atendimento.

Agradecemos por compartilhar sua experiência conosco, pois isso nos ajuda a melhorar nossos serviços. Esperamos ter a oportunidade de recebê-la novamente em nosso restaurante para proporcionar uma experiência mais satisfatória.

Atenciosamente,
A equipe do r

In [8]:
comment='Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!'

inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!
###
Caro cliente, agradecemos pelo seu feedback e lamentamos que sua experiência em nosso restaurante não tenha sido satisfatória. Levaremos em consideração suas observações sobre a falta de profissionalismo no nosso karaokê

In [9]:
comment = '''Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.'''
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.
###
Caro cliente, agradecemos pelo seu feedback e lamentamos pela má experiência que você teve em relação à higiene do nosso restaurante. Levaremos suas preocupações a sério e iremos reforçar a importância do mantenimento da limpeza e higiene. Vamos verificar a organização interna e reforçar o treinamento da nossa equipe para garantir que esses detalhes sejam corrigidos. Agradecemos pelo seu apreciado comentário e esperamos ter a oportunidade de recebê-lo novamente em nosso estabelecimento. Abraços, a no

In [10]:
comment = '''Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele é cubano.
Ia fazer uma denuncia por assedio mas pensei na empresa. Por favor selecione melhor seus colaboradores.
Muito indignada.'''

inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele é cubano.